In [2]:
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle 
import re 
from collections import defaultdict
import datetime
import codecs

In [3]:
# This method is done for you. Make sure to put your credentials in twitter.cfg.
def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [4]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request is None:
            print 'Found None object'
            break
        if request.status_code == 401:
            print 'Unauthorized data access. The user does not have access to such information'
            break
#         elif "page does not exist" in request.text:
#             print 'page does not exist'
#             break
        elif request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 1 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 1)

In [5]:
def get_followers():
    followers = []
    f = open ('iit_followers.txt','rb')
    regexp = re.compile(r'p\d+')
    for line in f:
        if regexp.search(str(line)) is not None:
            continue
        else: 
            if line.startswith('aV'):
                line = line[2:]
                followers.append(line.replace("\n",""))
    followers.sort()
    return followers

followers_list = get_followers()
#print followers_list

In [6]:
userInfo = twitter.request('users/lookup', {'screen_name': followers_list[:100]})

In [7]:
#get first 100 user IDs
userIds = []
startIndex = 0
endIndex = 100
while endIndex < len(followers_list):
    userInfo = twitter.request('users/lookup', {'screen_name': followers_list[startIndex:endIndex]})
    for user in userInfo:
        userIds.append(user['id'])
    startIndex += 100
    endIndex += 100
    print 'processed 100 users...'
print 'done'
print len(userIds)

processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 1

In [16]:
userInfo = twitter.request('users/lookup', {'screen_name': 'hpandya2_pa'})
for user in userInfo:
    userIds.insert(0, user['id'])
    print user['id']

2885816063


In [8]:
print userIds[:10]
print followers_list.index('ceoshea773')

[2958842221, 548263433, 3228267492, 476820795, 3397863503, 3158504957, 2786277625, 454864520, 117805662, 255609739]
3778


In [ ]:
def get_users():
    print 'in get_users()'
    #request = robust_request(twitter,'user', {'follow': followers_list})
    #request = twitter.request('user', {'follow': followers_list})
    #request2 = twitter.request('statuses/filter', {'follow':['stephenfgrove', 'stephniawang', 'sterlingeng']})
    request = twitter.request('statuses/filter', {'follow':userIds[:5000], 'filter_level':'low'})
    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    newfile = 'streaming_user_tweets_' + timestamp + '.txt'
    f = codecs.open(newfile,'a',encoding='utf8')
    tweets = 0
    print 'starting collection!!! :)'
    while True:
        try:
            for tweet in request:
                #print "saved tweet %d from %s" % (tweets, tweet['user']['name'])
                pickle.dump(tweet,f)
                f.write('\n')
                print 'dumped tweet to file'
                print tweet['text']
                tweets += 1
                if tweets > 10000:
                    tweets = 1
                    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                    newfile = 'streaming_user_tweets_' + timestamp + '.txt' 
        except:
            print >> sys.stderr, 'Got error: (sleeping for 60 seconds...)', request.text
            sys.stderr.flush()
            time.sleep(60) 
    
                    
get_users()               

in get_users()
starting collection!!! :)